In [19]:
import json, collections, os, random, glob, math, string, re, torch
import numpy as np
import timeit
from tqdm import trange, tqdm_notebook as tqdm 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from transformers import WEIGHTS_NAME, BertConfig, BertForQuestionAnswering, BertTokenizerFast, BasicTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers.models.bert.tokenization_bert import whitespace_tokenize
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor
from transformers import RobertaConfig, RobertaForQuestionAnswering, RobertaTokenizer
from transformers import XLMRobertaConfig, XLMRobertaForQuestionAnswering, XLMRobertaTokenizer
from transformers import AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer

In [20]:
class SquadExample(object):
    """
    A single training/test example for the Squad dataset.
    For examples without an answer, the start and end position are -1.
    """

    def __init__(self,
                 qas_id,
                 question_text,
                 doc_tokens,
                 orig_answer_text=None,
                 start_position=None,
                 end_position=None,
                 is_impossible=None,
                 answers=None):
        self.qas_id = qas_id
        self.question_text = question_text
        self.doc_tokens = doc_tokens
        self.orig_answer_text = orig_answer_text
        self.start_position = start_position
        self.end_position = end_position
        self.is_impossible = is_impossible
        self.answers = answers

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = ""
        s += "qas_id: %s" % (self.qas_id)
        s += ", question_text: %s" % (
            self.question_text)
        s += ", doc_tokens: [%s]" % (" ".join(self.doc_tokens))
        if self.start_position:
            s += ", start_position: %d" % (self.start_position)
        if self.start_position:
            s += ", end_position: %d" % (self.end_position)
        if self.start_position:
            s += ", is_impossible: %r" % (self.is_impossible)
        if self.start_position:
            s += ", answers: %r" % (self.answers)
        return s

In [42]:
def read_squad_examples(input_file, is_training):
    """Read a SQuAD json file into a list of SquadExample."""

    with open(input_file, "r", encoding='utf-8') as reader:
        source = json.load(reader)
        input_data = source["data"]
        version = source["version"]

    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False
    examples = []
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            paragraph_text = paragraph["context"].lower()
            doc_tokens = []
            char_to_word_offset = []
            prev_is_whitespace = True
            for c in paragraph_text:
                if is_whitespace(c):
                    prev_is_whitespace = True
                else:
                    if prev_is_whitespace:
                        doc_tokens.append(c)
                    else:
                        doc_tokens[-1] += c
                    prev_is_whitespace = False
                char_to_word_offset.append(len(doc_tokens) - 1)

            for qa in paragraph["qas"]:
                qas_id = qa["id"]
                question_text = qa["question"]
                start_position = None
                end_position = None
                orig_answer_text = None
                is_impossible = False
                answers = []
                if is_training:
                    if version == "v2.0":
                        is_impossible = qa["is_impossible"]
                    if (len(qa["answers"]) != 1) and (not is_impossible):
                        #print(entry["title"], qas_id)
                        raise ValueError(
                            "For training, each question should have exactly 1 answer.")
                    if not is_impossible:
                        answer = qa["answers"][0]
                        orig_answer_text = answer["text"].lower()
                        answer_offset = answer["answer_start"]
                        answer_length = len(orig_answer_text)
                        start_position = char_to_word_offset[answer_offset]
                        end_position = char_to_word_offset[answer_offset + answer_length - 1]
                        # Only add answers where the text can be exactly recovered from the
                        # document. If this CAN'T happen it's likely due to weird Unicode
                        # stuff so we will just skip the example.
                        #
                        # Note that this means for training mode, every example is NOT
                        # guaranteed to be preserved.
                        actual_text = " ".join(doc_tokens[start_position:(end_position + 1)])
                        cleaned_answer_text = " ".join(whitespace_tokenize(orig_answer_text))
                        if actual_text.find(cleaned_answer_text) == -1:
                            print("Could not find answer: '%s' vs. '%s'",
                                            actual_text, cleaned_answer_text)
                            continue
                    else: 
                        start_position = -1
                        end_position = -1
                        orig_answer_text = ""
                else:
                    answers = qa["answers"]
                example = SquadExample(
                    qas_id=qas_id,
                    question_text=question_text,
                    doc_tokens=doc_tokens,
                    orig_answer_text=orig_answer_text,
                    start_position=start_position,
                    end_position=end_position,
                    is_impossible=is_impossible,
                    answers=answers)
                
                #print(example)
                #type(example)
                #qas_id: uit_01__08947_13_1, 
                #question_text: Dân tộc nào có nhiều nét tương đồng nhất với dân tộc Ê Đê?, 
                #doc_tokens: [Người Ê đê và người Gia Rai vốn cùng nguồn gốc từ một tộc người Orang Đê cổ được ghi chép khá nhiều trong các bia ký Champa, Khmer,...Orang Đê có thể là nhóm mà người Ê đê và Gia Rai gọi là Mdhur, trong văn hóa Mdhur có chứa đựng nhiều yếu tố văn hoá trung gian giữa người Ê đê và Gia Rai. Trong văn hóa Mdhur trước kia còn tồn tại tục hỏa táng người chết và bỏ tro trong chum, ché sau đó mới mang chôn cất trong nhà mồ, đây có thể là ảnh hưởng của đạo Hin-đu từ người Chăm. Xét về phương diện người Mdhur là cội nguồn xuất phát của người Ê đê và Gia Rai hiện đại. Trong lịch sử Orang Đê đã từng tồn tại các tiểu quốc sơ khai, với sự cai trị của các Mtao, Pơ Tao có thế lực trên một khu vực rộng lớn ở vùng người Gia Rai và Ê đê. Sự hình thành các tiểu quốc nhỏ là đặc điểm thường thấy ở các tộc người Đông Nam á:], 
                #start_position: 5, 
                #end_position: 6, 
                #is_impossible: False, 
                #answers: []

                examples.append(example)
                

                #Ranking examples
    return examples


In [43]:
data = read_squad_examples(r'D:\NLP_project\data\SQuA2.0\train-v2.0-vi.json', True)

In [44]:
data[0]

qas_id: 0-1, question_text: Beyonce bắt đầu nổi tiếng từ khi nào?, doc_tokens: [beyoncé giselle knowles-carter (/ b i gì ɒ n s eɪ / bee-yon-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người mỹ. sinh ra và lớn lên ở houston, texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ r & b destiny's child. được quản lý bởi cha cô, mathew knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của beyoncé, dangerously in love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải grammy và có đĩa đơn quán quân billboard hot 100 "crazy in love" và "baby boy".], start_position: 55, end_position: 56, is_impossible: False, answers: []

In [24]:
input_file = r'D:\NLP_project\data\SQuA2.0\train-v2.0.json'
with open(input_file, "r", encoding='utf-8') as reader:
    source = json.load(reader)

In [33]:
source['data'][0]['paragraphs'][0]['qas'][0]['is_impossible']

False

In [45]:
input_file = r'D:\NLP_project\data\SQuA2.0\test-v2.0-preprocessed.json'
with open(input_file, "r", encoding='utf-8') as reader:
    org_source = json.load(reader)

In [46]:
idx = -1
data = {'version': 'v2.0', 'data': []}
text = None
for i, passage in enumerate(org_source):
    txt = passage[0]
    qus = passage[1]
    st_pos = txt.lower().find(passage[2].lower())
    ans = passage[2] if st_pos != -1 else ''
    is_impossible = st_pos == -1
    id = str(i) + str(idx)
    if txt != text:
        data['data'].append({'title': str(i), 'paragraphs': [
            {'qas':[{'question': qus, 'id': id, 'answers': [{'text': ans, 'answer_start': st_pos}], 'is_impossible': is_impossible}], 'context':txt}]})
        idx += 1
        text = txt
    else:
        data['data'][idx]['paragraphs'][0]['qas'].append({'question': qus, 'id': id, 'answers': [{'text': ans, 'answer_start': st_pos}], 'is_impossible': is_impossible})


In [27]:
data['data'][0]['paragraphs'][0]['qas'][2].keys()

dict_keys(['question', 'id', 'answers', 'is_impossible'])

In [47]:
with open("test-v2.0-vi.json", "w") as outfile:
    json.dump(data, outfile)